# v3.4

In [1]:
#@title Imports { display-mode: "form" }
from time import time as time_now
from time import sleep as time_sleep

import requests
from bs4 import BeautifulSoup

import os
import google.colab as gc

!pip install img2pdf
import img2pdf

! pip install aiohttp

from IPython.core.display import HTML, clear_output
from IPython.display import Image,display
from pprint import pprint

do_clear_output = True #@param {type:"boolean"}
if do_clear_output:
  clear_output()
else:pass

In [36]:
!git clone -b CrazyMetoFQ-asyn_lib https://github.com/CrazyMetoFQ/mscra.git "mscra2"

Cloning into 'mscra2'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 156 (delta 42), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (156/156), 117.36 KiB | 2.44 MiB/s, done.
Resolving deltas: 100% (42/42), done.


## MAIN



In [ ]:
start_time = time_now()
#@title Asyncrous Requests (supa fast multi) THIS IS THE MAIN STUFF TO RUN { display-mode: "form" }
#@markdown ---
#@markdown #Main Stuff:
#@markdown >Get the link by going to the websites first chapter then remove the ch number and replace with {}. <br>
#@markdown >Min, max are the number of chapters u want
# base_link where the number will change in the link such as ch-1, ch-2 put {}
base_link = "https://read-onepiecemanga.com/manga/one-piece-chapter-{}/" #@param {type:"string"}

# putting numbers in the link
min_ =  1 #@param {type:"number"}
max_ =   3#@param {type:"number"}
single_ = False #@param {type:"boolean"}

#@markdown *↓ use this to change file type (u can natively view img) ↓*
save_file_as = "image" #@param ["pdf", "image"]
save_file_format = ".jpg" #@param [".pdf", ".jpeg", ".jpg", ".png"] {allow-input: true}

#@markdown ---

clear_at_end = False #@param {type:"boolean"}
clear_wait = 0 #@param {type:"slider", min:0, max:15, step:1}



#@markdown ---
#@markdown <br> <br>

#@markdown ##Advanced Stuff
#@markdown <br>

# path to where the pdfs will reside
ch_path = "/content/chps" #@param {type:"string"}
#@markdown >><b><i> This is pretty important as it is ```[start:stop:step]``` for the list of imgs </i></b>, (image_args)
imag_args = "0,None,1" #@param {type:"string"}

do_zip = False #@param {type:"boolean"}
do_download = False #@param {type:"boolean"}
do_delete_prev = True #@param {type:"boolean"}

round_to = 3 #@param {type:"slider", min:1, max:10, step:1}

do_print_args = True #@param {type:"boolean"}
do_print_args = str(do_print_args)

zip_file_name = "file.zip" #@param ["/content/file.zip", "file.zip", "file2.zip"] {allow-input: true}

do_debug = False #@param {type:"boolean"}
img_key = "src" #@param ["src", "data-src", "lazy-loading-src"] {allow-input: true}



# correcting path
if not os.path.isdir(ch_path):
  os.mkdir(ch_path)
else:
  # deleting previos stuff
  if do_delete_prev:
    for fn in os.listdir(ch_path):
      try:
        os.remove(f"{ch_path}/{fn}")
      except:
        for sfn in os.listdir(f"{ch_path}/{fn}"):
          os.remove(f"{ch_path}/{fn}/{sfn}")
            
        os.removedirs(f"{ch_path}/{fn}")
      
    os.mkdir(ch_path)



# MAIN calling
! python /content/mscra/rasyn.py $base_link $min_ $max_ $single_ $ch_path $save_file_as $save_file_format $do_print_args $round_to $imag_args $do_debug $img_key



ch_end_time = time_now()
print(f"\nChapter Execution Time {round(ch_end_time - start_time, round_to)}")


if do_zip:
  print("\n-----------------------------\nNow Zipping")
  
  # zipping
  ! zip -r file.zip $ch_path
  
  zip_end_time = time_now()
  print(f"\nZip Execution Time {round(zip_end_time - start_time, round_to)}")
else:
  pass

if do_download:
  if os.path.isfile("file.zip"):
    # downloading
    gc.files.download("file.zip")
  else:
    print("Do Zip First")

  dw_end_time = time_now()
  print(f"\nDownload Init Time {round(dw_end_time - start_time, round_to)}")
else:
  pass

if clear_at_end:
  time_sleep(clear_wait)
  clear_output()
else:pass

print("You did It")
end_time = time_now()
print(f"\nTotal Execution time {round(end_time - start_time, round_to)}")

python3: can't open file 'rasyn.py': [Errno 2] No such file or directory

Chapter Execution Time 0.118
You did It

Total Execution time 0.119



---


It will now take some to download,my code is fast but colab download is slow
<br>
but if u go in files in folder chps u can individually download a pdf which is faster if u just want to see one chapter.
<hr>
(btw dont try spam downloading otherwise your browser might block it as spam download)

---



## Image View

In [ ]:
#@title image class { form-width: "10%", display-mode: "form" }
class ImageViewEr:

  def __init__(self,image_path):

    self.img_path = image_path
    self.img_list = [f"{self.img_path}/{img}" for img in os.listdir(self.img_path)]

  def sort_ls(self, x, y):
    """
    truly sorts image list

    x ->  stars wih
    y -> ends with
    """

    imd = {}
    for img in self.img_list:

      no = int(img[ img.find(x) + len(x) : img.find(y) ])
      imd[no] = img
          
    sorted_chs = [imd[k] for k in sorted(imd)]

    self.sorted_images = sorted_chs
    return sorted_chs 


  def format_imgs(self):
    """
    converts the image of path to img objs,
    and adds breaks between each image
    """

    imdi = []
    for im in self.sorted_images:

      imdi.append(Image(im))
      imdi.append(HTML('<div> <br>---------------------------<br> <br> </div>'))

    self.formated_images = imdi

  
  def show_images(self):

    display(HTML("""
    <style>
    #output-body {
        display: flex;
        align-items: center;
        justify-content: center;
        flex-direction: column;
    }
    </style>
    """))
    display(*self.formated_images)


chn = 0

In [ ]:
#@title Run This { display-mode: "form" }
da_im = ImageViewEr(f"/content/chps/ch_{chn}")

da_im.sort_ls("img",".jpg")
da_im.format_imgs()

da_im.show_images()

chn += 1

## Debugs

In [ ]:
#@title Debugger Advanced {display-mode:"form"}
#@markdown *to finding more about the page*
#@markdown <hr>

d_link = "https://psutil.readthedocs.io/en/latest/" #@param {type:"string"}

d_r = requests.get(d_link)
d_soup = BeautifulSoup(d_r.content, "html.parser")


def view_page():
  display(HTML(d_r.text))

def findAll_imgs():
  return d_soup.findAll('img')

def findAll_links():
  return d_soup.findAll('a')

def see_the_html():
  return d_soup.prettify()


want_view_page = True #@param {type:"boolean"}
want_findAll_imgs = False #@param {type:"boolean"}
want_findAll_a = False #@param {type:"boolean"}
want_see_da_html = False #@param {type:"boolean"}

choices_Dict = \
{
 "view_page":(want_view_page,view_page),
 "findAll_imgs":(want_findAll_imgs, findAll_imgs),
 "findAll_a":(want_findAll_a, findAll_links),
 "see_da_html":(want_see_da_html,see_the_html)
 }

for ch in choices_Dict:

  if choices_Dict[ch][0]:
    print(ch, "\n\n")
    pprint(choices_Dict[ch][1]())
    print("-"*15,'\n\n')
  else:
    pass
